# Endpoint test

In [1]:
from openai import OpenAI

client = OpenAI(
    api_key = "ocigenerativeai",
    base_url = "http://127.0.0.1:8088/api/v1/",
    )
models = client.models.list()

# Test chat completions
for model in models:
    print(model.id)


cohere.command-r-plus-08-2024
cohere.command-r-08-2024
cohere.command-r-plus
cohere.command-r-16k
meta.llama-3.3-70b-instruct
meta.llama-3.2-11b-vision-instruct
meta.llama-3.2-90b-vision-instruct
my-dedicated-model-name
my-datascience-model-name
ODSC-Mistral-7B-Instruct
ODSC-DeepSeek-R1-Distill-Qwen-7B
ODSC-sql-coder-graph


In [2]:
test_models = [
    "cohere.command-r-plus-08-2024",
    "cohere.command-r-08-2024",
    "meta.llama-3.3-70b-instruct",
    "meta.llama-3.2-90b-vision-instruct",
    ]


### Test chat completions with non-streaming response

In [3]:
# Test chat completions with non-streaming response
message = "Hello!"
print("User:", message)

for model_name in test_models:
    print("*"*20,"Model:",model_name,"*"*20,"\nAssistant:", end='')
    completion = client.chat.completions.create(
        model = model_name,
        messages = [{"role": "user", "content": message}],
        max_tokens=12,
        )
    print(completion.choices[0].message.content)

User: Hello!
******************** Model: cohere.command-r-plus-08-2024 ******************** 
Assistant:Hello! How can I help you today?
******************** Model: cohere.command-r-08-2024 ******************** 
Assistant:Hello! How can I assist you today?
******************** Model: meta.llama-3.3-70b-instruct ******************** 
Assistant:Hello. It's nice to meet you. Is there something
******************** Model: meta.llama-3.2-90b-vision-instruct ******************** 
Assistant:Hello. How can I assist you today?


### Test chat completions with streaming response

In [4]:
message = "Hello!"
print("User:", message)

for model in test_models:    
    print('\n',"*"*20,"Model:",model,"*"*20,"\nAssistant:", end='')
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': message}],
        max_tokens=12,
        stream=True  # this time, we set stream=True
    )
    for chunk in response:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content,end='')

User: Hello!

 ******************** Model: cohere.command-r-plus-08-2024 ******************** 
Assistant:Hello! How can I help you today?
 ******************** Model: cohere.command-r-08-2024 ******************** 
Assistant:Hello! How can I assist you today?
 ******************** Model: meta.llama-3.3-70b-instruct ******************** 
Assistant:Hello. How can I help you today?
 ******************** Model: meta.llama-3.2-90b-vision-instruct ******************** 
Assistant:Hello. How can I help you today.

### Test multi-modal with image input response

In [5]:
# Test multi-modal with image input response
import base64
import mimetypes

image_path = "image.jpg"
mime_type, _ = mimetypes.guess_type(image_path)
with open(image_path, "rb") as image_file:
    base64_str = base64.b64encode(image_file.read()).decode('utf-8')
url = f"data:{mime_type};base64,{base64_str}"
print(url[:100])

content = [
    {"type": "text","text": "describe this image?"},
	{"type": "image_url","image_url": {"url": url}}
    ]

test_models = ["meta.llama-3.2-90b-vision-instruct"]

for model in test_models:
    print('\n',"*"*20,"Model:",model,"*"*20)
    print("Model:", model)
    completion = client.chat.completions.create(
        model = model,
        messages = [{"role": "user", "content": content}],
        max_tokens=120,
        )
    print("Assistant:\n", completion.choices[0].message.content)

data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAeAB4AAD/4QAiRXhpZgAATU0AKgAAAAgAAQESAAMAAAABAAEAAAAAAAD/2

 ******************** Model: meta.llama-3.2-90b-vision-instruct ********************
Model: meta.llama-3.2-90b-vision-instruct
Assistant:
 The image features the iconic Superman logo, a red shield with a yellow "S" in the center and a black outline. The shield is diamond-shaped, with two triangular shapes on either side of the "S," resembling wings. The background of the image is a solid blue color.

**Key Elements:**

* **Logo:** A red shield with a yellow "S" in the center and a black outline.
* **Shield Shape:** Diamond-shaped with two triangular shapes on either side of the "S," resembling wings.
* **Background:** A solid blue color.

**Overall Impression:**




### Test tool calls

In [6]:
def get_weather(latitude, longitude):
    import requests
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]
model = "cohere.command-r-plus-08-2024"
# model = "cohere.command-r-plus-08-2024"
messages = [{"role": "user", "content": "What is the weather like in Paris and New York today?"}]
completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    stream=False
)

messages.append(completion.choices[0].message)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='eyJuYW1lIjogImdldF93ZWF0aGVyIiwgImFyZ3VtZW50cyI6ICJ7XCJsYXRpdHVkZVwiOiA0OC44NTY2LCBcImxvbmdpdHVkZVwiOiAyLjM1MjJ9In0=', function=Function(arguments='{"latitude": 48.8566, "longitude": 2.3522}', name='get_weather'), type='function', index=0), ChatCompletionMessageToolCall(id='eyJuYW1lIjogImdldF93ZWF0aGVyIiwgImFyZ3VtZW50cyI6ICJ7XCJsYXRpdHVkZVwiOiA0MC43MTI4LCBcImxvbmdpdHVkZVwiOiA3NC4wMDZ9In0=', function=Function(arguments='{"latitude": 40.7128, "longitude": 74.006}', name='get_weather'), type='function', index=1)]


In [7]:
# use tool to get weather
import json
for tool_call in completion.choices[0].message.tool_calls:
    args = json.loads(tool_call.function.arguments)
    result = get_weather(args["latitude"], args["longitude"])
    print(result)
    # append result message
    messages.append({                               
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(result)
    })

8.4
-4.2


In [8]:
# Generate answer based on tool results without stream
completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=False
)

print(completion_2.choices[0].message.content)

It's currently 8.4°C in Paris and -4.2°C in New York.


In [9]:
# Generate answer based on tool results with stream
completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True
    )
for chunk in completion_2:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='')

Today, the temperature in Paris is 8.4 degrees Celsius, and in New York it is -4.2 degrees Celsius.

### Test embedding

In [10]:
embd_model=[ 'cohere.embed-english-light-v3.0',
            'cohere.embed-english-v3.0',
            'cohere.embed-multilingual-light-v3.0',
            'cohere.embed-multilingual-v3.0']
input = ["hello!","你好！"]
for model in embd_model:
    print("model:",model)
    response = client.embeddings.create(input = input, model=model).data
    for each in response:
        print(each.index,':',str(each.embedding)[:36],'......',str(each.embedding)[-36:])
    print("*"*100)

model: cohere.embed-english-light-v3.0
0 : [0.049591064453125, 0.03173828125, - ...... 0824737548828125, 0.021087646484375]
1 : [-0.06787109375, -0.05828857421875,  ...... .0199127197265625, 0.04034423828125]
****************************************************************************************************
model: cohere.embed-english-v3.0
0 : [-0.015960693359375, 0.0127182006835 ...... -0.0203857421875, 0.020050048828125]
1 : [-0.01410675048828125, -0.0110168457 ...... 021575927734375, 0.0208587646484375]
****************************************************************************************************
model: cohere.embed-multilingual-light-v3.0
0 : [-0.018890380859375, 0.0327453613281 ...... 398193359375, 0.0008997917175292969]
1 : [0.038238525390625, 0.01398468017578 ......  0.08721923828125, 0.02984619140625]
****************************************************************************************************
model: cohere.embed-multilingual-v3.0
0 : [-0.00600433349609375, 0.0306